In [1]:
## Preloads
using Statistics
using FFTW
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using MLDatasets
using Images
theme(:juno)
using DSP
using Interpolations
using Distributed
using ProgressMeter

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
train_angles =range(π/48,stop=π,length=48)

0.06544984694978735:0.06544984694978735:3.141592653589793

In [3]:
test_angles = range(π/400,stop=π,length=400)

0.007853981633974483:0.007853981633974483:3.141592653589793

In [4]:
workers()

1-element Array{Int64,1}:
 1

In [5]:
addprocs(30)

30-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [6]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [7]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,6,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_eq_L6.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_train_eq_L6.h5", "main/angles", collect(train_angles))

Progress: 100%|█████████████████████████████████████████| Time: 2:31:27


In [7]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,6,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_eq_L6.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_eq_L6.h5", "main/angles", collect(test_angles))

Progress: 100%|█████████████████████████████████████████| Time: 4:08:28


In [8]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_eq_L4.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_train_eq_L4.h5", "main/angles", collect(train_angles))

Worker 30 terminated.
Worker 9 terminated.
Worker 11 terminated.
Worker 16 terminated.
Worker 28 terminated.
Worker 26 terminated.
Worker 4 terminated.
Worker 7 terminated.
Worker 5 terminated.
Worker 31 terminated.
Worker 18 terminated.
Progress: 100%|█████████████████████████████████████████| Time: 1:49:01


In [9]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(test_angles,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_eq_L4.h5", "main/data", mnist_DHC_out)
h5write("mnist_DHC_test_eq_L4.h5", "main/angles", collect(test_angles))

Progress:   0%|                                         |  ETA: 3:55:05

RemoteException: InterruptException:
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:851
deserialize_fillarray!(::Array{Any,1}, ::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1094
deserialize_array(::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1086
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:773
deserialize at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:722 [inlined]
deserialize_msg(::Distributed.ClusterSerializer{Sockets.TCPSock

In [26]:
test_angles = range(π/2,stop=π,length=2)

1.5707963267948966:1.5707963267948966:3.141592653589793

In [36]:
iter_prod0 = Iterators.product(test_angles,lst_test)
n_batch = 10
batch_size = length(iter_prod0)÷n_batch

2000

In [38]:
fid = h5open("mnist_DHC_test_eq_L4t.h5","w")
dset = create_dataset(fid, "main/data", datatype(Float64), dataspace(464,length(iter_prod0)), chunk=(464,batch_size),deflate=3)
close(fid)
@showprogress for i=1:n_batch
    subset = Iterators.take(iter_prod0,batch_size)
    mnist_DHC_out = @showprogress pmap(mnist_DHC,subset)
    mnist_DHC_out = hcat(mnist_DHC_out...)
    
    fid = h5open("mnist_DHC_test_eq_L4t.h5","r+")
    data = open_dataset(fid, "main/data")
    data[:,(1:batch_size).+(i-1)*batch_size] = mnist_DHC_out
    close(fid)
    iter_prod = Iterators.drop(iter_prod0,batch_size)
end

A = h5read("mnist_DHC_test_eq_L4t.h5", "main/data")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:29


464×20000 Array{Float64,2}:
 0.0176681    0.0176681    …  0.0457864    0.0180999   0.0180999 
 0.0113856    0.0113856       0.0363635    0.0108857   0.0108857 
 0.00245271   0.00171096      0.000651025  0.00345358  0.00445926
 0.0437785    0.0366561       0.00689771   0.0360503   0.0491932 
 0.0798014    0.0823397       0.0226704    0.0577736   0.103824  
 0.0573299    0.0615093    …  0.0411166    0.020275    0.0489461 
 0.0264434    0.0492241       0.0968289    0.0444225   0.061056  
 0.00263151   0.00153217      0.000876667  0.00409793  0.00381491
 0.0553846    0.0250503       0.00899767   0.0427346   0.0425088 
 0.108537     0.0536045       0.0314142    0.076301    0.0852957 
 0.0713657    0.0474736    …  0.0430769    0.0309674   0.0382534 
 0.0407415    0.0349258       0.0983894    0.0448401   0.0606381 
 0.00171097   0.00245272      0.00123329   0.00445927  0.00345358
 ⋮                         ⋱                                     
 0.036084     0.0302766       0.0825219    0.036

Ok, so the new flow is to call a wrapper that does the above

In [56]:
test_angles = range(π/400,stop=π,length=400)

0.007853981633974483:0.007853981633974483:3.141592653589793

In [7]:
function paraWrapper(filename,dataname,coeffdim,iter_prod,batch_size,n_batch)
    fid = h5open(filename,"w")
    dset = create_dataset(fid, dataname, datatype(Float64), dataspace(coeffdim,length(iter_prod)), chunk=(coeffdim,batch_size),deflate=3)
    close(fid)
    @showprogress for i=1:n_batch
        subset = Iterators.take(iter_prod,batch_size)
        DHC_out = pmap(mnist_DHC,subset)
        DHC_out = hcat(DHC_out...)

        fid = h5open(filename,"r+")
        data = open_dataset(fid, dataname)
        data[:,(1:batch_size).+(i-1)*batch_size] = DHC_out
        close(fid)
        iter_prod = Iterators.drop(iter_prod,batch_size)
    end
end

paraWrapper (generic function with 1 method)

In [9]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 50
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,4,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

80000

In [ ]:
paraWrapper("mnist_DHC_test_eq_L4.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L4.h5", "main/angles", collect(test_angles))

Progress:  36%|██████████████▊                          |  ETA: 0:59:06

In [ ]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_test_eq_L8.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L8.h5", "main/angles", collect(test_angles))

In [ ]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_train_eq_L8.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L8.h5", "main/angles", collect(train_angles))

In [ ]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,10,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_test_eq_L10.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L10.h5", "main/angles", collect(test_angles))

In [ ]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,10,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_train_eq_L10.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L10.h5", "main/angles", collect(train_angles))

In [ ]:
iter_prod = Iterators.product(test_angles,lst_test)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,12,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_test_eq_L12.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_test_eq_L12.h5", "main/angles", collect(test_angles))

In [ ]:
iter_prod = Iterators.product(train_angles,lst_train)
n_batch = 100
batch_size = length(iter_prod)÷n_batch

@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,12,nx=128,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end

coeffdim = size(mnist_DHC(collect(Iterators.take(iter_prod,1))[1]))[1]
batch_size

In [ ]:
paraWrapper("mnist_DHC_train_eq_L12.h5", "main/data",coeffdim,iter_prod,batch_size,n_batch)

h5write("mnist_DHC_train_eq_L12.h5", "main/angles", collect(train_angles))